In [2]:
# 17954936
import scanpy as sc
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
from scipy import sparse
import itertools as it
import sympy

In [3]:
FILE = "datasets/10XGenomics/Targeted_SC3v3_Human_Glioblastoma_Neuroscience_filtered_feature_bc_matrix.h5"
adata = sc.read_10x_h5(FILE)
X = adata.X.todense()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [4]:
a = Counter(np.array(X).flatten())
len(a)

942

In [16]:
B(942)

10

In [7]:
sum(1 for i in a.keys() if not sympy.isprime(int(i)))

790

In [4]:
adata.X

<5697x1186 sparse matrix of type '<class 'numpy.float32'>'
	with 1850701 stored elements in Compressed Sparse Row format>

In [18]:
np.product(X)

0.0

In [218]:
size = 2
x, y = X.shape
products = []
for i in tqdm.trange(0, y, 2):
    for j in range(0, x, 2):
        # print("matrix at", i, j)
        M = X[i:i+2, j:j+2]
        Mnz = M[M != 0]
        if Mnz.shape[1] == 0:
            products.append(0)
        else:
            products.append(np.prod(Mnz))

100%|█████████████████████████████████████████| 593/593 [00:08<00:00, 69.11it/s]


In [219]:
Counter(products)

Counter({0: 1467036,
         1.0: 68345,
         2.0: 34437,
         3.0: 17148,
         4.0: 13728,
         6.0: 8718,
         5.0: 6220,
         8.0: 5210,
         12.0: 4262,
         10.0: 3175,
         9.0: 3069,
         7.0: 3048,
         16.0: 2058,
         18.0: 2034,
         24.0: 1905,
         20.0: 1704,
         15.0: 1682,
         14.0: 1647,
         30.0: 1248,
         36.0: 1135,
         11.0: 1100,
         28.0: 887,
         48.0: 875,
         21.0: 870,
         40.0: 850,
         13.0: 818,
         32.0: 788,
         60.0: 682,
         42.0: 658,
         22.0: 625,
         27.0: 586,
         72.0: 585,
         45.0: 475,
         25.0: 460,
         54.0: 452,
         56.0: 448,
         26.0: 415,
         17.0: 415,
         84.0: 389,
         35.0: 387,
         90.0: 376,
         120.0: 376,
         19.0: 370,
         96.0: 360,
         44.0: 360,
         33.0: 357,
         80.0: 333,
         50.0: 319,
         64.0: 316,
   

In [81]:
1467036 * 2 + 68345 * 6 + 64777 * (6 + 8 + 2) + 89299 + (6 + 4 * 12)

4469927

In [59]:
sum(1 for i in products if not(i == 0 or i == 1 or sympy.isprime(int(i))))

89299

In [49]:
222421 * 4 + len(products)

2579141

In [50]:
max(products)

50274000.0

In [54]:
max_prime = max(i for i in products if sympy.isprime(int(i)))
max_nonprime = max(i for i in products if not sympy.isprime(int(i)))
max_prime, max_nonprime

(1063.0, 50274000.0)

In [6]:
def get_prime_index(i):
    j = 1
    while True:
        if sympy.prime(j) == i:
            return j
        j += 1

In [74]:
sympy.prime(179)

1063

In [80]:
len(bin(179)) - 2, len(bin(1063)) - 2, len(bin(2551)) - 2

(8, 11, 12)

In [79]:
np.max(X)

2551.0

In [77]:
sum(1 for i in products if sympy.isprime(int(i)))

64777

In [87]:
get_prime_index(1063)

179

In [91]:
prime_map = {sympy.prime(i): i for i in range(1, 179 + 1)}

In [8]:
def get_class(M, m=0):
    # row-wise classification
    a, b, c, d = M[0, 0] > m, M[0, 1] > m, M[1, 0] > m, M[1, 1] > m
    return (((((a << 1) + b) << 1) + c) << 1) + d

In [140]:
size = 2
x, y = X.shape
products = []
for i in tqdm.trange(0, y, 2):
    for j in range(0, x, 2):
        M = X[j:j+2, i:i+2]
        if M.shape != (2, 2):
            continue
        Mnz = M[M != 0]
        if Mnz.shape[1] == 0:
            products.append(0)
        else:
            products.append(np.prod(Mnz))

100%|█████████████████████████████████████████| 593/593 [00:13<00:00, 44.45it/s]


In [142]:
def write_bytes(handle, bits, bytes=4):
    handle.write(int(bits[::-1], 2).to_bytes(bytes, 'little'))
    return bytes * 4

def write_int(handle, value, bytes=4):
    handle.write(value.to_bytes(bytes, 'little'))
    return bytes * 4

def write_array(handle, array):
    bits = len(bin(int(max(array)))) - 2
    write_int(handle, len(array))
    write_int(handle, bits)

    # print(len(array), bits, (len(array) * 23) - (len(array) * bits + 2 * 32 * 8))
    curr_str = ""
    for elem in array:
        s = bin(int(elem))[2:].zfill(bits)
        for char in s:
            curr_str += char
            if len(curr_str) == 32:
                write_bytes(handle, curr_str)
                curr_str = ""
    if curr_str:
        write_bytes(handle, curr_str)

    return len(array) * bits + 2 * 32 * 8

In [ ]:
size = 2
x, y = X.shape
products = []
for i in tqdm.trange(0, y, 2):
    for j in range(0, x, 2):
        M = X[j:j+2, i:i+2]
        if M.shape != (2, 2):
            continue
        Mnz = M[M != 0]
        if Mnz.shape[1] == 0:
            products.append(0)
        else:
            products.append(np.prod(Mnz))

In [4]:
B = lambda i: len(bin(i)) - 2

class Multiplier:
    def __init__(self, handle, X, size=2):
        self.handle = handle
        self.X = X
        self.bits = ""
        self.products = []
        self.matrices = []
        self.size = size
        self.max_prime_index_bits = 0
        self.max_composite_bits = 0
        self.prime_map = {}
        self.total = 0
        self.f = None
        self.counts = defaultdict(int)

    def initialize(self):
        self.products = []
        self.matrices = []
        
        x, y = self.X.shape
        for i in tqdm.trange(0, y, self.size):
            for j in range(0, x, self.size):
                M = X[j:j+self.size, i:i+self.size]
                if M.shape != (self.size, self.size): # out of bounds error; todo handle odd columns
                    continue
                    
                Mnz = M[M != 0]
                if Mnz.shape[1] == 0:
                    self.products.append(0)
                else:
                    self.products.append(int(np.prod(Mnz)))
                self.matrices.append(M)
        
        max_prime = max(i for i in self.products if sympy.isprime(int(i)))
        max_prime_index = get_prime_index(max_prime)
        self.max_prime_index_bits = B(max_prime_index)
        self.prime_map = {sympy.prime(i): i for i in range(1, max_prime_index + 1)}

        self.max_composite_bits = B(int(np.max(self.X)))
        print("[+] initialized")
        return self

    def clear(self):
        self.bits = ""
        self.total = 0
        self.counts = defaultdict(int)

    def add_bits(self, bits):
        # print("adding", bits)
        for bit in bits:
            self.bits += bit
            self.flush()
        return len(bits)

    def _write_bytes(self, bits, bytes=4):
        self.f.write(int(bits[::-1], 2).to_bytes(bytes, 'little'))

    def _write_int(self, value, bytes=4):
        self.f.write(value.to_bytes(bytes, 'little'))

    def flush(self):
        if len(self.bits) != 32:
            return

        self._write_bytes(self.bits)
        self.bits = ""
        self.total += 32

    def get_matrix_bits(self, M, minimum=0):
        # row-wise classification
        # todo - not size-agnostic
        a, b, c, d = M[0, 0] > minimum, M[0, 1] > minimum, M[1, 0] > minimum, M[1, 1] > minimum
        return f"{int(a)}{int(b)}{int(c)}{int(d)}"

    def get_composite_matrix_bits(self, M):
        arr = M[M > 1]
        b = ""
        for i in range(arr.shape[1]):
            b += bin(int(arr[0, i]))[2:].zfill(self.max_composite_bits)
        return b

    def find_prime(self, M):
        m = self.get_matrix_bits(M, minimum=1)
        return bin(m.index('1'))[2:].zfill(2)
        
    def write(self):
        self.clear()
        with open(self.handle, "wb") as f:
            self.f = f
            for index, product in tqdm.tqdm(list(enumerate(self.products))):
                mclass = self.get_matrix_bits(self.matrices[index])
                if product == 0:
                    # print("A")
                    T = self.add_bits("00")
                    self.counts[0] += T
                elif product == 1:
                    # print("B")
                    T = self.add_bits("01")
                    # print("C")
                    T += self.add_bits(mclass)
                    self.counts[1] += T
                elif sympy.isprime(product):
                    # print("D")
                    T = self.add_bits("10")
                    # print("E")
                    T += self.add_bits(mclass)
                    # print("F")
                    T += self.add_bits(self.find_prime(self.matrices[index]))
                    # print("G")
                    T += self.add_bits(bin(product)[2:].zfill(self.max_prime_index_bits))
                    self.counts[2] += T
                else:
                    # print("H")
                    T = self.add_bits("11")
                    # print("I")
                    T += self.add_bits(mclass)
                    # print("J")
                    T += self.add_bits(self.get_matrix_bits(self.matrices[index], minimum=1))
                    # print("K")
                    T += self.add_bits(self.get_composite_matrix_bits(self.matrices[index]))
                    m = B(int(np.max(self.matrices[index])))
                    n = len(self.get_composite_matrix_bits(self.matrices[index])) / self.max_composite_bits
                    self.counts[3] += T
                    self.counts['3W'] += (n * self.max_composite_bits) - (n * m)
            return self.total, self.counts

In [ ]:
1467036 * 2 + 68345 * 6 + 64777 * (6 + 8 + 2) + 89299 + (6 + 4 * 12)

In [19]:
13768710 / sum(1 for i in m.products if not (i == 0 or i == 1 or sympy.isprime(i)))

32.714176758640846

In [237]:
(32.7 - 6) / 13

2.053846153846154

In [234]:
4976499 / sum(1 for i in m.products if sympy.isprime(i))

16.000524080367562

In [235]:
1994724 / sum(1 for i in m.products if i == 1)

6.0

In [236]:
1249020 / sum(1 for i in m.products if i == 0)

2.0

In [20]:
sum(1 for i in m.products if not (i == 0 or i == 1 or sympy.isprime(i)))

420879

In [7]:
m = Multiplier("targ.mult", X)
m.initialize()
m.write()

100%|█████████████████████████████████████████| 593/593 [00:08<00:00, 73.31it/s]


[+] initialized


100%|█████████████████████████████| 1688864/1688864 [00:16<00:00, 102079.91it/s]


(21988928,
 defaultdict(int,
             {1: 1994724,
              3: 13768710,
              '3W': 6381900.0,
              2: 4976499,
              0: 1249020}))

In [23]:
len(Counter(m.products))

9896

In [24]:
B(max(m.products))

27

In [25]:
9896 * 27

267192

In [8]:
len(m.matrices) - 624510

1064354

In [10]:
len(m.matrices) + 8 * (len(m.matrices) - 624510)

10203696

In [27]:
B(9896)

14

In [17]:
Counter(map(m.get_matrix_bits, m.matrices))

Counter({'0000': 624510,
         '0100': 139190,
         '0001': 135627,
         '0101': 126738,
         '1000': 120108,
         '0010': 117503,
         '1010': 97046,
         '1100': 52765,
         '0011': 51240,
         '1111': 50214,
         '1101': 37804,
         '0111': 36763,
         '1110': 29842,
         '1011': 29317,
         '1001': 20111,
         '0110': 20086})

In [21]:
b = Counter(map(lambda i: tuple(np.array(i).flatten()), m.matrices))

In [22]:
len(b)

73522

In [238]:
get_class(np.array([[0, 0], [0, 0]]))

0

In [203]:
bin(0)[2:].zfill(2)

'00'

In [175]:
import itertools
i = 38
j = 40
M = X[i:i+2, j:j+2]
M[M > 1][0, 0]

2.0